## Import libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import os
import matplotlib.pyplot as plt

In [3]:
print(torch.cuda.is_available())

True


If the above output **False** means you can't use cuda acceleration. Don't worry It'll still work fine.

## Define CNN

Network structure:

Conv2d -> Activation -> MaxPooling -> Conv2d -> Activation -> MaxPooling -> Flattening -> FullyConnected

Used function:
- Convolution: nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
- Activation: nn.ReLU()
- MaxPooling: nn.MaxPool2d(kernel_size)
- FullyConnected: nn.Linear(input_channels, output_channels)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 28, 28)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # input shape (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 14, 14)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output, x    # return x for visualization

In [5]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


## Preparing Dataset

Check if MNIST is already downloaded

In [6]:
DOWNLOAD_MNIST = False
# Mnist digits dataset
if not(os.path.exists('./dataset/MNIST/')) or not os.listdir('./dataset/MNIST/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True
    
print(DOWNLOAD_MNIST)

False


Loading training data

In [7]:
train_data = torchvision.datasets.MNIST(
    root='./dataset/',
    train=True,                                     # this is training data
    transform=torchvision.transforms.ToTensor(),    # Converts a PIL.Image or numpy.ndarray to
                                                    # torch.FloatTensor of shape (C x H x W) and normalize in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,
)
# Data Loader for easy mini-batch return in training, the image batch shape will be (50, 1, 28, 28)
train_loader = Data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

pick 2000 samples to speed up testing

In [ ]:
test_data = torchvision.datasets.MNIST(root='./dataset/', train=False)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.test_labels[:2000]

plot one example

In [ ]:
PIC_IDX = 4 # change this to see different picture
print(train_data.train_data.size())                 # (60000, 28, 28)
print(train_data.train_labels.size())               # (60000)
plt.imshow(train_data.train_data[PIC_IDX].numpy(), cmap='gray')
plt.title('%i' % train_data.train_labels[PIC_IDX])
plt.show()

## Training
Now we are ready to train

In [ ]:
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
LR = 0.001              # learning rate

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

In [ ]:
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):   # 分配 batch data, normalize x when iterate train_loader
        output, hidden = cnn(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        if step % 50 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| step: ', step, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
    

print 10 predictions from test data

In [ ]:
test_output, _ = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')